In [434]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Set display options to show all columns
pd.set_option('display.max_columns', None)
np.random.seed(42)  # You can use any integer value as the seed

CWD = os.getcwd()
print(f'CWD: {CWD}')

CWD: /data_analysis/IT_support/src


In [435]:
cardinal_cols = ['number', 'caller_id', 'opened_by', 'sys_created_by', 'sys_updated_by', 'contact_type', 'location', 'u_symptom']
ordinal_cols = ['incident_state', 'reassignment_count', 'reopen_count', 'sys_mod_count', 'impact', 'urgency', 'priority']

boolean_cols = ['cmdb_ci', 'u_priority_confirmation',]

dt_cols = ['opened_at', 'sys_created_at', 'sys_updated_at', 'closed_at']

# to_remove = ['made_sla', 'knowledge', 'notify', 'problem_id', 'rfc', 'vendor', 'caused_by', 'closed_code', 'resolved_by', 'resolved_at', 'closed_at',
#              'assignment_group', 'assigned_to', 'category', 'subcategory', 'active']

all_cols = cardinal_cols + ordinal_cols + boolean_cols + dt_cols
len(all_cols)

21

In [436]:
# Read in a fresh copy of the dataset
ROOT = os.path.dirname(CWD)
PATH = os.path.join(ROOT, 'data', 'incident_event_log.csv')
df = pd.read_csv(PATH, low_memory=False, na_values='?')

# Remove 'Closed' tickets and tickets with no 'Number'
df = df.loc[df['incident_state']!='Closed', all_cols]
df = df[~df['number'].isna()] # Remove entries where number is blank

for col in dt_cols:
    df[col] = pd.to_datetime(df[col], dayfirst=True)
df.sample(5)

,number,caller_id,opened_by,sys_created_by,sys_updated_by,contact_type,location,u_symptom,incident_state,reassignment_count,reopen_count,sys_mod_count,impact,urgency,priority,cmdb_ci,u_priority_confirmation,opened_at,sys_created_at,sys_updated_at,closed_at
34505,INC0007687,Caller 2677,Opened by 255,NaN,Updated by 775,Email,Location 197,Symptom 491,Resolved,1,0,4,2 - Medium,2 - Medium,3 - Moderate,NaN,True,2016-03-16 10:57:00,NaT,2016-03-22 12:55:00,2016-03-27 12:59:00
33668,INC0007500,Caller 5485,Opened by 8,Created by 6,Updated by 307,Phone,Location 143,Symptom 387,Active,2,0,6,2 - Medium,2 - Medium,3 - Moderate,NaN,False,2016-03-16 06:22:00,2016-03-16 06:41:00,2016-03-17 15:12:00,2016-03-24 19:01:00
10762,INC0002461,Caller 1293,Opened by 402,Created by 172,Updated by 868,Phone,Location 204,Symptom 473,Awaiting User Info,0,0,4,2 - Medium,2 - Medium,3 - Moderate,NaN,False,2016-03-04 11:57:00,2016-03-04 12:02:00,2016-03-14 12:28:00,2016-03-24 18:50:00
15077,INC0003384,Caller 1309,Opened by 70,Created by 47,Updated by 777,Phone,Location 55,Symptom 494,New,1,0,3,2 - Medium,2 - Medium,3 - Moderate,NaN,False,2016-03-07 11:45:00,2016-03-07 11:47:00,2016-03-15 16:04:00,2016-03-24 18:59:00
64955,INC0015190,Caller 2410,Opened by 441,Created by 182,Updated by 783,Phone,Location 204,Symptom 93,Resolved,0,0,1,2 - Medium,2 - Medium,3 - Moderate,NaN,True,2016-04-05 06:58:00,2016-04-05 07:00:00,2016-04-05 08:15:00,2016-04-10 08:59:00


In [437]:
# Create the target column
df['update_to_closing_days'] = df['closed_at']-df['sys_updated_at']
df['update_to_closing_days'] = df['update_to_closing_days'].dt.days + df['update_to_closing_days'].dt.seconds / (24 * 60 * 60)

df.drop('closed_at', axis=1, inplace=True)
df = df[df['opened_at']<df['sys_updated_at']]
df = df[~df['update_to_closing_days'].isna()]

## Feature Engineering
Create new features using Sklearn `FunctionTransformer` this way everything can be put together in the pipeline.

In [438]:
from sklearn.preprocessing import FunctionTransformer
def ticket_age(X):
    X['ticket_age_open'] = (X['sys_updated_at'] - X['opened_at']).dt.total_seconds() / (3600 * 24) # Days since ticket was opened
    X['ticket_age_create'] = (X['sys_updated_at'] - X['sys_created_at']).dt.total_seconds() / (3600 * 24) # Days since ticket was created in system
    X['days_before_sys_create'] = (X['opened_at'] - X['sys_created_at']).dt.total_seconds() / (3600 * 24) # Gap between opening and creation in system

    # Compute average ticket resolution time per priority
    avg_resolve_time = X[X['incident_state']=='Resolved'].groupby('priority')['ticket_age_create'].mean().reset_index()
    X = pd.merge(X, avg_resolve_time, on='priority', how='left', suffixes=('', '_mean'))

    # Compute average ticket age at each stage
    avg_age = X.groupby('incident_state')['ticket_age_create'].mean().reset_index()
    X = pd.merge(X, avg_age, on='incident_state', how='left', suffixes=('', '_mean'))

    return X

ticket_age_transform = FunctionTransformer(ticket_age)


In [439]:
# Models cannot handle datetime format by itself. Split datetime components to separate components.
def split_dt_parts(X):
    cols = X.select_dtypes(include=['datetime64']).columns.to_list()
    for col in cols:
        for component in ['year', 'month', 'day', 'hour', 'minute', 'second']:
            new_col_name = f'{col}_{component}'  # Create a dynamic column name
            X[new_col_name] = X[col].dt.__getattribute__(component).astype(float)
    return X

split_dt_parts_transform = FunctionTransformer(split_dt_parts)

# Drop the original datetime columns and the columns used for computations
def drop_columns(X):
    cols = X.select_dtypes(include=['datetime64']).columns.to_list()
    cols+=['number', 'incident_state','priority']
    X.drop(columns=cols, inplace=True)
    return X

drop_columns_transform = FunctionTransformer(drop_columns)

### Pipeline

In [440]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.feature_selection import SelectPercentile, chi2, SelectKBest
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso


from sklearn.metrics import mean_squared_error

In [441]:
# Create the column transformer

cardinal_cols = ['contact_type', 'location', 'u_symptom']
ordinal_cols = ['incident_state', 'sys_mod_count', 'priority',]
boolean_cols = ['cmdb_ci', 'u_priority_confirmation',]
dt_cols = ['opened_at', 'sys_created_at', 'sys_updated_at', 'number', 'incident_state', 'priority'] # Number & incident_state are here for feature engineering. will be dropped later.

categorical_features = cardinal_cols + ordinal_cols + boolean_cols
datetime_features = dt_cols

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    # ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1))
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

datetime_transformer = Pipeline(steps=[
    ('compute_ticket_age', ticket_age_transform),
    # ('split_dt_parts', split_dt_parts_transform),
    ('drop_cols', drop_columns_transform),
    ('imputer', SimpleImputer(strategy='constant', fill_value=-1.0, add_indicator=True)),
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('datetime', datetime_transformer, datetime_features),
    ]
)

# Create the final pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', XGBRegressor())
                           ])

pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['contact_type', 'location',
                                                   'u_symptom',
                                                   'incident_state',
                                                   'sys_mod_count', 'priority',
                                                   'cmdb_ci',
                                                   'u_priority_confirmation']),
                                                 ('datetime',
                                                  Pipeline(steps=[('compute...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

### Train Test Split

In [442]:
# Define the features and target
df.sort_values(['opened_at', 'number'])

# Define the groups based on the 'number' column
groups = df['number']

# Get unique group values
unique_groups = groups.unique()
split_idx = int(0.8 * len(unique_groups))

# train_num, test_num = train_test_split(unique_groups, test_size = 0.2, random_state=42)
train_num, test_num = unique_groups[:split_idx], unique_groups[split_idx:]
train = df[df['number'].isin(train_num)]
test = df[df['number'].isin(test_num)]

X_train, y_train = train.drop(columns=['update_to_closing_days',]), train['update_to_closing_days']
X_test, y_test = test.drop(columns=['update_to_closing_days',]), test['update_to_closing_days']

### Naive Predictions

In [443]:
# Compute the mean and standard deviation of y_train
mean_y_train = np.mean(y_train)
std_dev_y_train = np.std(y_train)

# Generate y_preds using random normal distribution
y_pred = np.random.normal(loc=mean_y_train, scale=std_dev_y_train, size=len(y_test))

# Calculate the mean squared error (MSE)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Root Mean Squared Error (MSE): {rmse}")

Root Mean Squared Error (MSE): 30.87627605392623


### Model Prediction

In [444]:
# Fit the pipeline to the training data
pipeline.fit(X_train, y_train)

# Make predictions on the test data
y_pred = pipeline.predict(X_test)

# Calculate the mean squared error (MSE)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Root Mean Squared Error (MSE): {rmse}")

Root Mean Squared Error (MSE): 21.326383838935048


### Fine tuning

In [445]:
from sklearn.model_selection import RandomizedSearchCV, GroupKFold

# Define the number of folds for cross-validation
n_splits = 5  # You can adjust this as needed

# Create a GroupKFold cross-validator using the 'number' column as groups
group_kfold = GroupKFold(n_splits=n_splits)

# Define the hyperparameter grid for XGBoostRegressor
param_grid_xgb = {
    'model': [XGBRegressor()],
    # 'model__n_estimators': np.linspace(100, 500, 5, dtype=int),
    'model__learning_rate': np.logspace(-3, 0, 20),
    # 'model__max_depth': np.linspace(3, 10, 8, dtype=int),
    # 'model__min_child_weight': np.linspace(1, 5, 5, dtype=int),
    # 'model__subsample': np.linspace(0.8, 1.0, 3),
    # 'model__colsample_bytree': np.linspace(0.8, 1.0, 3),
    'model__gamma': np.linspace(0,5,50)
}

param_grid_linear = {
    'model': [LinearRegression()],
}

param_grid_rf = {
    'model': [RandomForestRegressor()],
    'model__n_estimators': [50, 100, 200],
    'model__max_depth': [None, 10, 20, 30],
    'model__min_samples_split': [2, 5, 10],
    'model__min_samples_leaf': [1, 2, 4],
}

param_grid_lasso = {
    'model': [Lasso()],
    'model__alpha': [0.01, 0.1, 1.0],
    'model__normalize': [False, True],
}

all_params = [param_grid_xgb, param_grid_rf, param_grid_linear, param_grid_lasso]

# Create the RandomizedSearchCV object with GroupKFold
random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=all_params,
    n_iter=10,
    scoring='neg_mean_squared_error',
    cv=group_kfold,  # Use GroupKFold for cross-validation
    n_jobs=-1,
    verbose=1,
    random_state=42,
)

# Fit the RandomizedSearchCV object on your data
random_search.fit(X_train, y_train, groups=X_train['number'])  # Pass the 'number' column as groups

# Get the best hyperparameters and estimator from the search
best_params = random_search.best_params_
best_estimator = random_search.best_estimator_

# Print the best hyperparameters
print(f'Best Hyperparameters: {best_params}')

# Make predictions on the test data
y_pred = best_estimator.predict(X_test)

# Calculate the mean squared error (MSE)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Root Mean Squared Error (MSE): {rmse}")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best Hyperparameters: {'model__learning_rate': 0.05455594781168517, 'model__gamma': 4.387755102040816, 'model': XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=4.387755102040816, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05455594781168517,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)}
Root Mean Squared Error (MSE): 21.237428986820124
